# Assignment 2 - Recurrent Neural Networks



## Programming (Full points: 100)

In this assignment, our goal is to use PyTorch to implement Recurrent Neural Networks (RNN) for sentiment analysis task. Sentiment analysis is to classify sentences (input) into certain sentiments (output labels), which includes positive, negative and neutral.

We will use a benckmark dataset, SST, for this assignment.
* we download the SST dataset from torchtext package, and do some preprocessing to build vocabulary and split the dataset into training/validation/test sets. You don't need to modify the code in this step.


In [1]:
import copy
import torch
from torch import nn
from torch import optim
import torchtext
from torchtext import data
from torchtext import datasets

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.LabelField()

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL)

# build dictionary
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

# hyperparameters
vocab_size = len(TEXT.vocab)
label_size = len(LABEL.vocab)
padding_idx = TEXT.vocab.stoi['<pad>']
embedding_dim = 128
hidden_dim = 128

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size=32)

downloading trainDevTestTrees_PTB.zip


.data\sst\trainDevTestTrees_PTB.zip: 100%|███████████████████████████████████████████| 790k/790k [00:00<00:00, 851kB/s]


extracting


* define the training and evaluation function in the cell below.
### (25 points)


In [2]:
# your code here
import torch
import torch.nn as nn
import torch.optim as optim

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for batch in dataloader:
        inputs, labels = batch.text.to(device), batch.label.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return total_loss / len(dataloader), accuracy

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch.text.to(device), batch.label.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return total_loss / len(dataloader), accuracy



* build a RNN model for sentiment analysis in the cell below.
We have provided several hyperparameters we needed for building the model, including vocabulary size (vocab_size), the word embedding dimension (embedding_dim), the hidden layer dimension (hidden_dim), the number of layers (num_layers) and the number of sentence labels (label_size). Please fill in the missing codes, and implement a RNN model.
### (40 points)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the RNN model class
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, label_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, label_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        out = self.fc(output[:, -1, :])
        return out

* train the model and compute the accuracy in the cell below.
### (20 points)

In [7]:
# Define hyperparameters
vocab_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 128
num_layers = 2
label_size = len(LABEL.vocab)
learning_rate = 0.001
epochs = 10

# Create the model and move it to GPU if available
model = RNNModel(vocab_size, embedding_dim, hidden_dim, num_layers, label_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and evaluate the model
for epoch in range(epochs):
    train_loss, train_accuracy = train(model, train_iter, optimizer, criterion, device)
    valid_loss, valid_accuracy = evaluate(model, val_iter, criterion, device)
    
    print(f'Epoch {epoch+1}:')
    print(f'Training Loss: {train_loss:.4f} | Training Accuracy: {train_accuracy:.2f}%')
    print(f'Validation Loss: {valid_loss:.4f} | Validation Accuracy: {valid_accuracy:.2f}%')

# Test the model on the test set
test_loss, test_accuracy = evaluate(model, test_iter, criterion, device)
print(f'Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%')


KeyboardInterrupt: 

* try to train a model with better accuracy in the cell below. For example, you can use different optimizers such as SGD and Adam. You can also compare different hyperparameters and model size.
### (15 points), to obtain FULL point in this problem, the accuracy needs to be higher than 70%